## Smoothing

Smoothing can help to discover trends that otherwise might be hard to see in raw data. 

In [1]:
%useLatestDescriptors
%use lets-plot
//%use krangl

In [2]:
LetsPlot.getInfo()  // This prevents Krangl from loading an obsolete version of Lets-Plot classes.

Lets-Plot Kotlin API v.4.1.1. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.2.5.1.

In [3]:
%use krangl

In [4]:
var mpg_df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/data/mpg.csv")
mpg_df.head()


,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


### The default smoothing method is `'linear model'` (or `'lm'`)

In [5]:
// val dat = (mpg_df.names.map { Pair(it, mpg_df.get(it).values())}).toMap()
val dat = mpg_df.toMap()

In [6]:
val mpg_plot = letsPlot(dat) {x="displ"; y="hwy"} 
mpg_plot + geomPoint() + geomSmooth()

### `LOESS` model does seem to better fit MPG data than linear model.

In [7]:
mpg_plot + geomPoint() + statSmooth(method="loess", size=1.0)

### Applying smoothing to groups

Let's map the vehicle `drivetrain type` (variable 'drv') to the color of points.

This makes it easy to see that points with the same type of the drivetrain are forming some kind of groups or clusters. 

In [8]:
mpg_plot + geomPoint {color="drv"} +
           statSmooth(method="loess", size=1.0) {color="drv"}

### Apply linear model with 2nd degree polynomial.

As `LOESS` prediction looks a bit weird let's try 2nd degree polinomial regression.

In [9]:
mpg_plot + geomPoint {color="drv"} +
           statSmooth(method="lm", deg=2, size=1.0) {color="drv"}

## Using `asDiscrete()` function with numeric data series

In the previous examples we were using a discrete (or categorical) variable `'drv'` to split the data into a groups.

Now let's try to use a numeric variable `'cyl'` for the same purpose.

In [10]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0) {color="cyl"}

Easy to see that the data wasn't split into groups. 
`Lets-Plot` offers two solutions in this situation:

 * Use the `group` aesthetic
 * Use the `asDiscrete()` function

The **group** aesthetic helps to create a groups.

In [11]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0) {color="cyl"; group="cyl"}

The **asDiscrete('cyl')** function will "annotate" the `'cyl'` variable as `discrete`.

This leads to creation of the groups and to assigning of a `discrete` color scale instead of a `continuous`.

In [12]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0) {color=asDiscrete("cyl")}

## Effect of `span` parameter on the "wiggliness" the LOESS smoother.

The span is the fraction of points used to fit each local regression.
Small numbers make a wigglier curve, larger numbers make a smoother curve.

In [13]:
import kotlin.math.PI
import kotlin.random.Random


In [14]:
val n = 150
val x_range = generateSequence( -2 * PI ) { it + 4 * PI / n }.takeWhile { it <= 2 * PI }
val y_range = x_range.map{ sin( it ) + Random.nextDouble(-0.5, 0.5) }
val df = mapOf(
    "x" to x_range,
    "y" to y_range
)

In [15]:
val p = ggplot(df) {x="x"; y="y"} + geomPoint(shape=21, fill="yellow", color="#8c564b")
val p1 = p + geomSmooth(method="loess", size=1.5, color="#d62728") + ggtitle("default (span = 0.5)")
val p2 = p + geomSmooth(method="loess", span=.2, size=1.5, color="#9467bd") + ggtitle("span = 0.2")
val p3 = p + geomSmooth(method="loess", span=.7, size=1.5, color="#1f77b4") + ggtitle("span = 0.7")
val p4 = p + geomSmooth(method="loess", span=1, size=1.5, color="#2ca02c") + ggtitle("span = 1")

GGBunch()
.addPlot(p1, 0, 0, 400, 300)
.addPlot(p2, 400, 0, 400, 300)
.addPlot(p3, 0, 300, 400, 300)
.addPlot(p4, 400, 300, 400, 300)